对4千条业务B数据做简单特征工程，采用XGBoost的方式获得模型A然后训练得到预测

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split,StratifiedKFold
from sklearn.metrics import auc,roc_auc_score
import xgboost as xgb

In [3]:
b_train = pd.read_csv('B_train_dummy.csv')
b_test =  pd.read_csv('B_test_dummy.csv') 

In [4]:
# 将train与test的列属性统一起来

col = [x for x in b_train.columns if x in b_test.columns]
col = [x for x in col if x not in ['no', 'flag']]


In [7]:
# 划分训练集与验证集

train_X,test_X,train_Y,test_Y = train_test_split(b_train[col], b_train['flag'],test_size=0.2, random_state=0)

# 获取模型的评价数据
watchlist = [(xgb.DMatrix(train_X,label=train_Y),'train'), (xgb.DMatrix(test_X,label=test_Y),'eval')]


In [10]:
dtrain_B = xgb.DMatrix(b_train[col], b_train['flag'])

# 线上效果为0.600018的参数
Trate=0.25 
params = {'booster':'gbtree',
              'eta': 0.05, 
              'max_depth': 4,                  
              'max_delta_step': 0,
              'subsample':1,              
              'colsample_bytree': 0.9,      
              'base_score': Trate, 
              'objective': 'binary:logistic', 
              'lambda':3,
              'alpha':5
              }
params['eval_metric'] = 'auc' 

# 训练模型
model_phase_1_cla_2 = xgb.train(params,dtrain_B,num_boost_round=130,maximize=True,verbose_eval=True) 

# 预测
pred = model_phase_1_cla_2.predict(xgb.DMatrix(test_X))
# 看看auc值
test_auc = roc_auc_score(test_Y, pred)
print(test_auc) # 线下0.7549725810948973


0.7549725810948973


In [11]:
# 得到测试集的预测值保存
pred_test = model_phase_1_cla_2.predict(xgb.DMatrix(b_test[col]))
result = pd.DataFrame()
result['no'] = b_test['no']
result['pred'] = pred_test[:]
result.to_csv('subimit_target.csv',index=False)

In [ ]:
# 线上auc：0.600018